### Exercise: Monte Carlo Pi on the GPU

Let's revisit Monte Carlo Pi generating algorithm from the first section, where we had compiled it with Numba on the CPU.

In [8]:
import numpy as np
from numba import cuda
from numba.cuda.random import xoroshiro128p_uniform_float32
import math

threads_per_block = 64
blocks = 24
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)

In [9]:
from numba import njit
import random

@njit
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [10]:
nsamples = 10000000
%timeit monte_carlo_pi(nsamples)

492 ms ± 9.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
from numba import njit
import random

# TODO: All your work will be in this cell. Refactor to run on the device successfully given the way the
# kernel is launched below.
@cuda.jit
def monte_carlo_pi_device(rng_states, samples, out):
    # Obtener el índice global único para este hilo
    # idx = cuda.grid(1) podría ser más idiomático aquí
    thread_id = cuda.grid(1) # Equivalente a cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x

    # Asegurarse de no exceder los límites del array rng_states o out
    # (Buena práctica, aunque en este caso grid_size coincide)
    if thread_id >= rng_states.shape[0]:
        return
        
    acc = 0
    for i in range(samples): # Usar el argumento 'samples' (samples_per_thread)
        # Generar números aleatorios usando el estado del hilo actual
        x = xoroshiro128p_uniform_float32(rng_states, thread_id)
        y = xoroshiro128p_uniform_float32(rng_states, thread_id)

        # Verificar si el punto está dentro del círculo unitario
        if x**2 + y**2 < 1.0:
            acc += 1

    # Escribir la proporción calculada por este hilo en el array de salida
    # Cada hilo calcula su propia estimación parcial de Pi / 4
    out[thread_id] = 4.0 * acc / samples # Escribir en el array 'out', NO retornar
  

In [12]:
# Do not change any of the values in this cell
nsamples = 10000000
threads_per_block = 128
blocks = 32

grid_size = threads_per_block * blocks
samples_per_thread = int(nsamples / grid_size) # Each thread only needs to work on a fraction of total number of samples.
                                               # This could also be calcuated inside the kernel definition using `gridsize(1)`.

rng_states = create_xoroshiro128p_states(grid_size, seed=1)
d_out = cuda.device_array(threads_per_block * blocks, dtype=np.float32)

In [14]:
%timeit monte_carlo_pi_device[blocks, threads_per_block](rng_states, samples_per_thread, d_out); cuda.synchronize()

C:\Users\santi\miniconda3\envs\env_numba\lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


11 ms ± 152 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
# Celda después de llamar al kernel y sincronizar:
print(d_out.copy_to_host().mean())

3.1413999
